In [1]:
# -*- coding: utf-8 -*-
import ansys.fluent.core as pyfluent
from ansys.fluent.core import examples
from matplotlib import pyplot
import os, sys
import csv
import math
import pandas as pd
import numpy as np
import argparse

In [2]:
#启动fluent，载入网格，检查有无负体积网格

solver = pyfluent.launch_fluent(mode="solver",
                                version="3d",
                                precision="double",
                                processor_count=10,
                                show_gui=True)

solver.file.read(file_type="case", file_name="data/tps5_id6_x_t-new.cas")
solver.tui.mesh.check()

Reading "data/tps5_id6_x_t-new.cas"...
Buffering for file scan...



    7790 polyhedra cells, zone 69, binary.
    7790 cell partition ids, zone 69, 10 partitions, binary.
   45395 polygonal interior faces, zone 68, binary.
     201 polygonal wall faces, zone 40, binary.
     204 polygonal wall faces, zone 39, binary.
     193 polygonal wall faces, zone 38, binary.
     196 polygonal wall faces, zone 37, binary.
    1798 polygonal wall faces, zone 36, binary.
    1885 polygonal wall faces, zone 35, binary.
   44969 nodes, binary.
44969 node flags, binary.

Checking the existence of compiled "udf" UDF library for current platform.
If needed will attempt auto-compilation...

 	 Make sure that UDF source files are in the directory 
 	 that contains your case and data files. If you have 
 	 an existing udf directory, please remove this 
 	 directory to ensure that latest files are used. 
  

Error: Auto-compilation skipped. Could not find "udf" or input files for creating "udf" library
Er

In [3]:
solver.setup.models.energy.enabled = True
solver.tui.define.models.radiation.rosseland('yes')

In [ ]:
#加载UDF文件
solver.tui.define.user_defined.use_built_in_compiler("yes")  #启用udf编译器
solver.tui.define.user_defined.compiled_functions("compile", "udf", "yes", "y", "radiation.c")  #编译udf源文件radiation.c
solver.tui.define.user_defined.compiled_functions("load","udf")  #加载udf

In [4]:

solver.tui.solve.report_files.add("add", 'up-middle', 'activate', 'yes', 'print', 'yes', 'quit')
solver.tui.solve.report_files.edit("edit", 'up-middle', 'report-defs', '"flow_time"', (), 'file_name', "./up-middle1.csv" ,'quit')

The following solver settings object method could also be used to execute the above command:
<solver_session>.solution.monitor.report_files["add"] = {}


In [5]:
#solver.tui.define.materials.change_create('nano', 'nano', 'yes', 'constant', '370.', 'yes', 'constant', '700.', 'yes', 'constant', '0.03' )


parser = argparse.ArgumentParser()

parser.add_argument('--density', type=float, default=370., help='density')
parser.add_argument('--cp', type=float, default=700., help='cp')
parser.add_argument('--lamda', type=float, default=0.03, help='lamda')

parser.add_argument('--time_step_size', type=float, default=0.01, help='time_step_size')
parser.add_argument('--max_iterations_per_time_step', type=float, default=30, help='max_iterations_per_time_step')
parser.add_argument('--number_of_time_steps', type=float, default=30, help='number_of_time_steps')




args = parser.parse_args([])   #jupyter必须赋个初值

#物性参数
density = args.density
cp = args.cp
lamda = args.lamda

#iteration参数
time_step_size = args.time_step_size
max_iterations_per_time_step = args.max_iterations_per_time_step
number_of_time_steps = args.number_of_time_steps



solver.setup.materials.solid['nano'] = {'density': {'option': 'constant', 'value': density},
                                       'specific_heat': {'option': 'constant', 'value': cp},
                                        'thermal_conductivity': {'option': 'constant', 'value': lamda}}


heat_flux = [20000, 30000, 40000]

for index, q in enumerate(heat_flux):

    #solver.tui.define.boundary_conditions.wall('wall', 'up','0.01','no', '0.','no','yes', 'no', '{0}','no','no', '1' )
    solver.setup.boundary_conditions.wall['up'] = {'q': {'option': 'value', 'value': q}}
    

    #初始化：standard initialization
    #solver.tui.solve.initialize.compute_defaults.all_zones()
    solver.tui.solve.initialize.reference_frame("relative")
    solver.tui.solve.initialize.set_defaults('temperature','273.15')
    solver.tui.solve.initialize.initialize_flow('yes')  
    # solver.tui.solve.initialize.hyb_initialization()   hybrid initialization
    print('初始化结束')

    #开始计算
    solver.tui.define.models.unsteady_2nd_order()
    solver.tui.solve.set.transient_controls.time_step_size(time_step_size)
    solver.tui.solve.set.transient_controls.max_iterations_per_time_step(max_iterations_per_time_step)
    solver.tui.solve.set.transient_controls.number_of_time_steps(number_of_time_steps)
    solver.tui.solve.dual_time_iterate()

    #后处理

    
    #solver.tui.solve.dual_time_iterate('dual_time_iterate', '100', '20')
    solver.tui.file.write_case(('work/tps5_' + str(q)  + '.cas'))
    print('迭代完成')

初始化结束
The following solver settings object method could also be used to execute the above command:
<solver_session>.solution.run_calculation.dual_time_iterate(time_step_count = 30, max_iter_per_step = 30)
The following solver settings object method could also be used to execute the above command:
<solver_session>.file.write(file_type = "case", file_name = "tps5_20000.cas")
迭代完成
The following solver settings object method could also be used to execute the above command:
<solver_session>.solution.initialization.standard_initialize()
初始化结束
The following solver settings object method could also be used to execute the above command:
<solver_session>.solution.run_calculation.dual_time_iterate(time_step_count = 30, max_iter_per_step = 30)
The following solver settings object method could also be used to execute the above command:
<solver_session>.file.write(file_type = "case", file_name = "tps5_30000.cas")
迭代完成
The following solver settings object method could also be used to execute the abov

In [6]:
solver.exit()